In [23]:
import pyzx as zx
from pyzx import Graph, VertexType
from sympy import symbols, pi
from ZXLS import *

In [24]:
m1,m2,m3,m4 = symbols('m1,m2,m3,m4')
rm = Graph()
rs = Graph()
ss = Graph()
sm = Graph()
rough_merge = create_rough_merge(rm,0,1,0,m1)
smooth_split = create_smooth_split(ss,0,0,1,m2)
rough_split = create_rough_split(rs,0,0,1,m3)
smooth_merge = create_smooth_merge(sm,0,1,0,m4)
zx.draw(rm)
zx.draw(rs)
zx.draw(sm)
zx.draw(ss)

TypeError: create_rough_merge() missing 1 required positional argument: 'meas_outcome'

In [25]:
protocol = Graph()

in0 = protocol.add_vertex(VertexType.BOUNDARY,qubit=0,row=0)
out0 = protocol.add_vertex(VertexType.BOUNDARY,qubit=0,row=1)

in1 = protocol.add_vertex(VertexType.Z,qubit = 1,row=0)
out1 = protocol.add_vertex(VertexType.BOUNDARY, qubit=1,row=1)

in2 = protocol.add_vertex(VertexType.BOUNDARY,qubit=2,row=0)
out2 = protocol.add_vertex(VertexType.BOUNDARY,qubit=2,row=1)

protocol_edges = protocol.add_edges([(in0,out0),(in1,out1),(in2,out2)])
protocol.set_inputs([in0,in1,in2])
protocol.set_outputs([out0,out1,out2])
m1,m2,m3,m4,m5,m6 = symbols('m1,m2,m3,m4,m5,m6')

sm01 = Graph()
sm = create_smooth_merge(sm01,0,1,0,m1)
smin3 = sm01.add_vertex(VertexType.BOUNDARY,qubit=2,row=0)
smout3 = sm01.add_vertex(VertexType.BOUNDARY,qubit=2,row=3)
sm01.add_edge((smin3,smout3))
sm01.set_inputs([sm[0],sm[1],smin3])
sm01.set_outputs([sm[2],smout3])

zx.draw(protocol)
zx.draw(sm01)
protocol1 = protocol.compose(sm01)
zx.draw(protocol)


TypeError: create_smooth_merge() missing 1 required positional argument: 'meas_outcome'

In [65]:
import pyzx as zx
from pyzx import Graph, VertexType
from sympy import symbols, pi

def create_smooth_split(graph: Graph, inp: int, out1: int, out2: int, meas_outcome: symbols):
    """Create a smooth split (Z-spider) with X correction on upper output leg."""
    inp_vert = graph.add_vertex(VertexType.BOUNDARY, qubit=inp, row=0)
    split = graph.add_vertex(VertexType.Z, phase=0, qubit=inp, row=1)
    correction = graph.add_vertex(VertexType.X, phase=meas_outcome, qubit=out1, row=2)
    upper = graph.add_vertex(VertexType.BOUNDARY, qubit=out1, row=3)
    lower = graph.add_vertex(VertexType.BOUNDARY, qubit=out2, row=3)
    graph.add_edges([(inp_vert, split), (split, correction),
                     (correction, upper), (split, lower)])
    graph.set_inputs([inp_vert])
    graph.set_outputs([upper, lower])
    return upper, lower, inp_vert

def create_rough_split(graph: Graph, inp: int, out1: int, out2: int, meas_outcome: symbols):
    """Create a rough split (X-spider) with Z correction on upper output leg."""
    inp_vert = graph.add_vertex(VertexType.BOUNDARY, qubit=inp, row=0)
    split = graph.add_vertex(VertexType.X, phase=0, qubit=inp, row=1)
    correction = graph.add_vertex(VertexType.Z, phase=meas_outcome, qubit=out1, row=2)
    upper = graph.add_vertex(VertexType.BOUNDARY, qubit=out1, row=3)
    lower = graph.add_vertex(VertexType.BOUNDARY, qubit=out2, row=3)
    graph.add_edges([(inp_vert, split), (split, correction),
                     (correction, upper), (split, lower)])
    graph.set_inputs([inp_vert])
    graph.set_outputs([upper, lower])
    return upper, lower, inp_vert

def create_smooth_merge(graph: Graph, in1: int, in2: int, out: int, meas_outcome: symbols):
    """Create a smooth merge (Z-spider) with X correction on upper input leg."""
    inp1 = graph.add_vertex(VertexType.BOUNDARY, qubit=in1, row=0)
    inp2 = graph.add_vertex(VertexType.BOUNDARY, qubit=in2, row=0)
    correction = graph.add_vertex(VertexType.X, phase=meas_outcome, qubit=in1, row=1)
    merge = graph.add_vertex(VertexType.Z, phase=0, qubit=in1, row=2)
    out_vert = graph.add_vertex(VertexType.BOUNDARY, qubit=out, row=3)
    graph.add_edges([(inp1, correction), (correction, merge),
                     (inp2, merge), (merge, out_vert)])
    graph.set_inputs([inp1, inp2])
    graph.set_outputs([out_vert])
    return inp1, inp2, out_vert

def create_rough_merge(graph: Graph, in1: int, in2: int, out: int, meas_outcome: symbols):
    """Create a rough merge (X-spider) with Z correction on upper input leg."""
    inp1 = graph.add_vertex(VertexType.BOUNDARY, qubit=in1, row=0)
    inp2 = graph.add_vertex(VertexType.BOUNDARY, qubit=in2, row=0)
    correction = graph.add_vertex(VertexType.Z, phase=meas_outcome, qubit=in1, row=1)
    merge = graph.add_vertex(VertexType.X, phase=0, qubit=in1, row=2)
    out_vert = graph.add_vertex(VertexType.BOUNDARY, qubit=out, row=3)
    graph.add_edges([(inp1, correction), (correction, merge),
                     (inp2, merge), (merge, out_vert)])
    graph.set_inputs([inp1, inp2])
    graph.set_outputs([out_vert])
    return inp1, inp2, out_vert

# --- Code from start ---
protocol = Graph()

# Define initial wires/state preparation
in0 = protocol.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Control Input
out0 = protocol.add_vertex(VertexType.BOUNDARY, qubit=0, row=1)
in1_prep = protocol.add_vertex(VertexType.Z, qubit=1, row=0) # |0> state prep
out1 = protocol.add_vertex(VertexType.BOUNDARY, qubit=1, row=1)
in2 = protocol.add_vertex(VertexType.BOUNDARY, qubit=2, row=0) # Target Input
out2 = protocol.add_vertex(VertexType.BOUNDARY, qubit=2, row=1)
protocol.add_edges([(in0, out0), (in1_prep, out1), (in2, out2)])

protocol.set_inputs([in0, in2]) # Inputs are Control and Target
protocol.set_outputs([out0, out1, out2]) # Outputs are Control-wire, |0>-state, Target-wire
# Removed normalize

m1, m2, m3, m4, m5, m6 = symbols('m1,m2,m3,m4,m5,m6')

# Create graph for smooth merge of q0 and q1, plus q2 wire
sm01_g = Graph()
sm_in_out = create_smooth_merge(sm01_g, 0, 1, 0, m1) # Returns (in0_b, in1_b, out0_b)
sm01_in0, sm01_in1 = sm_in_out[0], sm_in_out[1]
sm01_out0 = sm_in_out[2]
# Add q2 wire
sm01_in2 = sm01_g.add_vertex(VertexType.BOUNDARY, qubit=2, row=0)
sm01_out2 = sm01_g.add_vertex(VertexType.BOUNDARY, qubit=2, row=3) # Align row
sm01_g.add_edges([(sm01_in2, sm01_out2)])
# Set overall inputs/outputs for this step (q0, q1, q2 -> q0, q2)
sm01_g.set_inputs([sm01_in0, sm01_in1, sm01_in2])
sm01_g.set_outputs([sm01_out0, sm01_out2])
# Removed normalize

print("Initial Protocol Graph:")
zx.draw(protocol)
print("\nSmooth Merge (q0, q1 -> q0) + Wire (q2) Graph:")
zx.draw(sm01_g)

# Compose the first merge step
protocol.compose(sm01_g)
# protocol graph now represents the state after the first merge.
# Its outputs are on qubit 0 (merged state) and qubit 2 (target wire).

print("\nProtocol after First Smooth Merge:")
zx.draw(protocol) # Draw graph state *after* compose

# --- Add the rest of the CNOT operations using compose ---

# 2. Create Smooth Split graph *including identity for qubit 2*
ss_graph = Graph()
# Apply split to input q0 -> outputs q0, q1
ss_out0_b, ss_out1_b, ss_in0_b = create_smooth_split(ss_graph, inp=0, out1=0, out2=1, meas_outcome=m2)
# Add identity wire for qubit 2
ss_in2_b = ss_graph.add_vertex(VertexType.BOUNDARY, qubit=2, row=0) # Input q2
ss_out2_b = ss_graph.add_vertex(VertexType.BOUNDARY, qubit=2, row=3) # Output q2 (align row)
ss_graph.add_edges([(ss_in2_b, ss_out2_b)])
# Set inputs (q0, q2) and outputs (q0, q1, q2) for compose
ss_graph.set_inputs([ss_in0_b, ss_in2_b])
ss_graph.set_outputs([ss_out0_b, ss_out1_b, ss_out2_b])
# Removed normalize

print("\nSmooth Split Graph (In: q0,q2 -> Out: q0,q1,q2):")
zx.draw(ss_graph)

# Compose the smooth split
# protocol outputs (q0, q2) match ss_graph inputs (q0, q2)
protocol.compose(ss_graph)
# protocol graph outputs are now on qubit 0 (final control), qubit 1 (ancilla), qubit 2 (target wire).

print("\nProtocol after Smooth Split:")
zx.draw(protocol) # Draw graph state *after* compose

# 3. Create Rough Merge graph *including identity for qubit 0*
rm_graph = Graph()
# Apply merge to inputs q1, q2 -> output q2
rm_in1_b, rm_in2_b, rm_out2_b = create_rough_merge(rm_graph, in1=1, in2=2, out=2, meas_outcome=m3)
# Add identity wire for qubit 0
rm_in0_b = rm_graph.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Input q0
rm_out0_b = rm_graph.add_vertex(VertexType.BOUNDARY, qubit=0, row=3) # Output q0 (align row)
rm_graph.add_edges([(rm_in0_b, rm_out0_b)])
# Set inputs (q0, q1, q2) and outputs (q0, q2) for compose
rm_graph.set_inputs([rm_in0_b, rm_in1_b, rm_in2_b])
rm_graph.set_outputs([rm_out0_b, rm_out2_b])


print("\nRough Merge Graph (In: q0,q1,q2 -> Out: q0,q2):")
zx.draw(rm_graph)

# Compose the rough merge
# protocol outputs (q0, q1, q2) match rm_graph inputs (q0, q1, q2)
protocol.compose(rm_graph)
# protocol graph outputs are now on qubit 0 (final control) and qubit 2 (final target)

print("\nFinal CNOT Protocol Graph:")
meas_ancilla=protocol.add_vertex(VertexType.Z,phase=m3,qubit=1,row=7)
protocol.add_edges([(in1_prep,meas_ancilla)])
zx.draw(protocol) # Draw final graph state


# Assuming 'protocol' is the final graph after all compositions and manual measurement addition

print("\nAttempting simplification...")
try:
    # IMPORTANT: Simplify a copy to preserve the original graph
    protocol_copy = protocol.copy()

    # Apply the main simplification routine
    #protocol_simplified = zx.simplify.full_reduce(protocol_copy)

    print("\nSimplified CNOT Graph:")
    zx.draw(protocol_copy)

    # You might need further simplification for presentation, but full_reduce does the bulk
    # e.g., converting graph-like forms, but likely not needed here.
    # protocol_simplified = zx.simplify.teleport_reduce(protocol_simplified)
    # zx.draw(protocol_simplified)

    zx.spider_simp(protocol_copy)
    zx.draw(protocol_copy)

    protocol_circuit = zx.extract_circuit(protocol_copy)
    print("Circuit gates: ", protocol_circuit.gates)
    zx.draw(protocol_circuit)

    print("\n--- Analysis of Simplified Graph ---")
    print(f"Number of vertices: {len(protocol_copy.vertices())}")
    print("Phases:", [protocol_copy.phase(v) for v in protocol_copy.vertices() if protocol_copy.phase(v) != 0])
    # Check structure: Should ideally have ~6-7 nodes for CNOT + phases

except Exception as e:
    print(f"\nSimplification failed: {e}")
    # Common reasons: Highly complex symbolic expressions, timeouts, bugs in specific rule applications.

Initial Protocol Graph:



Smooth Merge (q0, q1 -> q0) + Wire (q2) Graph:



Protocol after First Smooth Merge:



Smooth Split Graph (In: q0,q2 -> Out: q0,q1,q2):



Protocol after Smooth Split:



Rough Merge Graph (In: q0,q1,q2 -> Out: q0,q2):



Final CNOT Protocol Graph:



Attempting simplification...

Simplified CNOT Graph:


Circuit gates:  [HAD(1), ZPhase(0,phase=Mod(m1, 2)), HAD(0), HAD(1), CZ(0,1), ZPhase(0,phase=2*(Mod(Mod(m3, 2), 1))), HAD(0), ZPhase(0,phase=Mod(m2, 2))]



--- Analysis of Simplified Graph ---
Number of vertices: 4
Phases: []


In [8]:
from ZXLS import (Graph, PauliFrame, create_smooth_merge, create_smooth_split, 
                   substitute_measurements)
from sympy import symbols
from pyzx import draw, simplify
# Create a fresh graph and pauli frame for this test.
g_chain = Graph()
pf_chain = PauliFrame()
m1, m2 = symbols('m1 m2')

# Step 1: Smooth merge with symbolic outcome m1.
inp1, inp2, merge_out = create_smooth_merge(g_chain, 0, 1, 0, m1, pauli_frame=pf_chain)
print("After smooth merge, Pauli frame:", pf_chain)
print("Graph after smooth merge (count):", len(g_chain.vertices()))

# Step 2: Smooth split on the merged output (simulate splitting qubit 0 into qubits 0 and 1).
split_upper, split_lower, merge_in = create_smooth_split(g_chain, inp=0, out1=0, out2=1, meas_outcome=m2)
print("After smooth split, Graph vertex count:", len(g_chain.vertices()))

# Set overall inputs/outputs if needed, then substitute measurement outcomes.
# For instance, substitute m1=0, m2=1:
g_chain_sub = substitute_measurements(g_chain, {m1: 0, m2: 1})
print("After substitution, simplified graph vertex count:", len(g_chain_sub.vertices()))


After smooth merge, Pauli frame: PauliFrame({0: 'X'})
Graph after smooth merge (count): 5
After smooth split, Graph vertex count: 10
After substitution, simplified graph vertex count: 9


In [ ]:
draw(g_chain_sub)  # Visualize the final graph after all operations and substitutions.
new = simplify.full_reduce(g_chain_sub)  # Simplify the graph if needed.

In [17]:
draw(g_chain_sub)  # Visualize the simplified graph.

In [8]:
from ZXLS import *
import pyzx as zx

n = 5

ancillas = [
    {"qubit_number": 2, "basis": "Z", "phase": 0},
    {"qubit_number": 3, "basis": "X", "phase": 1}
]

parser = ParseInstructions(6, ancillas)

parser.add_operation("s-merge",2,3,2)
parser.add_operation("s-merge",0,1,0)
parser.add_operation("s-merge",0,2,0)

parser.add_operation("s-merge",4,5,4) # Debug this operation
parser.add_operation("r-merge",0,4,0)

parser.execute_operations()

## CNOT Protocol
# S-Merge (0,1,0)
# S-Plit (0,1,0)
# R-Merge (0,2,0)

zx.draw(parser.total_graph, labels=True)


In [3]:
from ZXLS import *
import pyzx as zx

ancillas = [
    {"qubit_number": 1, "basis": "Z", "phase": 0},
]

parser = ParseInstructions(3, ancillas)

parser.add_operation("s-merge",0,1,0)
parser.add_operation("s-split",0,0,1)
parser.add_operation("r-merge",1,2,2)

parser.execute_operations()

print("qubits: ", parser.total_graph.qubits())

zx.draw(parser.total_graph, labels=True)

qubits:  {0: 0, 2: 1, 4: 2, 6: 0, 7: 0, 8: 0, 9: 0, 10: 2, 11: 1, 12: 1, 13: 0}


In [4]:
zx.spider_simp(parser.total_graph)
zx.draw(parser.total_graph)

In [5]:
#parser.total_graph.normalize()
print(parser.total_graph.qubits())
zx.draw(parser.total_graph, labels=True)
circuit = zx.extract_circuit(parser.total_graph)

{0: 0, 4: 2, 6: 0, 7: 0, 9: 0, 10: 2, 12: 1, 13: 0}


In [55]:
circuit.gates

[SWAP(0,1),
 HAD(1),
 ZPhase(1,phase=Mod(m0, 2)),
 HAD(1),
 HAD(0),
 CZ(0,1),
 ZPhase(1,phase=Mod(m2, 2)),
 HAD(1),
 ZPhase(1,phase=Mod(m1, 2)),
 HAD(1),
 HAD(0)]

In [68]:
zx.draw(circuit, labels=True)

In [1]:
from ZXLS import *
import pyzx as zx
import io 

ancillas = [
    {"qubit_number": 1, "basis": "Z", "phase": 0},
]

parser = ParseInstructions(10, ancillas)

# 

parser.add_operation("s-merge",0,1,0)
#parser.add_operation("s-merge",1,2,1)
parser.add_operation("s-split",0,0,1)
#parser.add_operation("r-merge",1,2,2)

parser.execute_operations()

file_cont = parser.total_graph.to_json("test.json")
#save file to disk
file = open("test.json", "w")
file.write(file_cont)
file.close() 

zx.draw(parser.total_graph, labels=True)

In [34]:
parser2 = ParseInstructions(20)

subgraph4toLen = parser2.total_graph.subgraph_from_vertices([4,5,6,7,8,9,10,11])

print("Subgraph qubit 0: ", subgraph4toLen.qubit(0))

zx.draw(subgraph4toLen, labels=True)

ts = parser.total_graph.tensor(subgraph4toLen)

print("Tensor qubit 5: ", ts.qubit(5))

zx.draw(ts, labels=True)

Subgraph qubit 0:  2


Tensor qubit 5:  4


In [14]:
import json

# Open and read the JSON file
with open('test.json', 'r') as file:
    data = json.load(file)
    
g = parser.total_graph.from_json(data)
zx.draw(g, labels=True)

ValueError: Unexpected token Token('LPAR', '(') at line 1, column 4.
Expected one of: 
	* PLUS
	* $END
	* STAR
Previous tokens: [Token('CNAME', 'Mod')]
